# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
total_schools = school_data["school_name"].count()
total_students = school_data_complete["Student ID"].count()
total_budget = school_data["budget"].sum()
avg_math = school_data_complete["math_score"].mean()
avg_reading = school_data_complete["reading_score"].mean()
overall_rate = (avg_math + avg_reading)/2

passing_math = school_data_complete.loc[school_data_complete["math_score"]>=70,"math_score"]
per_passing_math = (passing_math.count()/total_students)*100

passing_reading = school_data_complete.loc[school_data_complete["reading_score"]>=70,"reading_score"]
per_passing_reading = (passing_reading.count()/total_students)*100

summary=[total_schools, total_students, total_budget, avg_math, avg_reading, per_passing_math, per_passing_reading, overall_rate]

district_summary = pd.DataFrame([summary])
district_summary = district_summary.rename(columns={0:"Total Schools",
                                                   1:"Total Students",
                                                   2:"Total Budget",
                                                   3:"Average Math Score",
                                                   4:"Average Reading Score",
                                                   5:"% Passing Math",
                                                   6:"% Passing Reading",
                                                   7:"% Overall Passing Rate"})
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [42]:
grpby_schl = school_data_complete.groupby(["school_name","type","budget"])


schl_summary_df = pd.DataFrame(
    grpby_schl.agg({'Student ID' : 'count'
                    , 'math_score' : 'mean'
                    , 'reading_score' : 'mean'
                   }))

schl_summary_df = schl_summary_df.reset_index()

schl_summary_df = schl_summary_df.rename(columns = {"Student ID" : "Total Students"
                           , "math_score" : "Average Math Score"
                           , "reading_score" : "Average Reading Score"
                           , "type" : "School Type"
                           , "budget" : "Total School Budget"})

schl_summary_df["Per Student Budget"] = schl_summary_df["Total School Budget"] // schl_summary_df["Total Students"]


#Filter out Scores >= 70, create dataframes for these
passed_math_grp = school_data_complete.loc[((school_data_complete["math_score"]) >= 70)
                                           ,["math_score", "school_name"]].groupby(["school_name"])
passed_math_df = pd.DataFrame(passed_math_grp.agg({'math_score' : 'count'}) )
passed_math_df = passed_math_df.reset_index()


passed_reading_grp = school_data_complete.loc[((school_data_complete["reading_score"]) >= 70)
                                           ,["reading_score", "school_name"]].groupby(["school_name"])
passed_reading_df = pd.DataFrame(passed_reading_grp.agg({'reading_score' : 'count'}) )
passed_reading_df = passed_reading_df.reset_index()


#Merge dfs created above to the main table
schl_summary_df = pd.merge(schl_summary_df, passed_math_df, on="school_name", how="left")
schl_summary_df = pd.merge(schl_summary_df, passed_reading_df, on="school_name", how="left")

schl_summary_df["math_score"] = (schl_summary_df["math_score"]/schl_summary_df["Total Students"])*100
schl_summary_df["reading_score"] = (schl_summary_df["reading_score"]/schl_summary_df["Total Students"])*100

schl_summary_df["% Overall Passing Rate"] = (schl_summary_df["math_score"] + schl_summary_df["reading_score"])/2


#rename and reorder columns
schl_summary_df = schl_summary_df.rename(columns = {"math_score" : "% Passing Math"
                           , "reading_score" : "% Passing Reading"})


schl_summary_df.set_index("school_name", inplace = True) 

schl_summary_df = schl_summary_df[["School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]

schl_summary_df.sort_values(by=["% Overall Passing Rate"], inplace = True, ascending=False)

top_schls_df = schl_summary_df.head(5)

top_schls_df


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,638,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,609,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,625,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [44]:
schl_summary_df.sort_values(by=["% Overall Passing Rate"], inplace = True, ascending=True)

bottom_schl_df = schl_summary_df.head(5)

bottom_schl_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,639,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,655,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,644,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [59]:
math_grade_9 = school_data_complete.loc[(school_data_complete["grade"]=="9th") ,["school_name", "math_score"]]
math_grade_10 = school_data_complete.loc[(school_data_complete["grade"]=="10th") ,["school_name", "math_score"]]
math_grade_11 = school_data_complete.loc[(school_data_complete["grade"]=="11th") ,["school_name", "math_score"]]
math_grade_12 = school_data_complete.loc[(school_data_complete["grade"]=="12th") ,["school_name", "math_score"]]

math_grade_9_grp = math_grade_9.groupby(["school_name"])
math_grade_9_grp = pd.DataFrame(math_grade_9_grp.agg({'math_score' : 'mean'})).reset_index()

math_grade_10_grp = math_grade_10.groupby(["school_name"])
math_grade_10_grp = pd.DataFrame(math_grade_10_grp.agg({'math_score' : 'mean'})).reset_index()

math_grade_11_grp = math_grade_11.groupby(["school_name"])
math_grade_11_grp = pd.DataFrame(math_grade_11_grp.agg({'math_score' : 'mean'})).reset_index()

math_grade_12_grp = math_grade_12.groupby(["school_name"])
math_grade_12_grp = pd.DataFrame(math_grade_12_grp.agg({'math_score' : 'mean'})).reset_index()
#math_grade_12_grp

math_by_grade_df = pd.merge(math_grade_9_grp, math_grade_10_grp, on="school_name")  
math_by_grade_df = math_by_grade_df.rename(columns={"math_score_x" : "9th"})
math_by_grade_df = math_by_grade_df.rename(columns={"math_score_y" : "10th"})

math_by_grade_df = pd.merge(math_by_grade_df, math_grade_11_grp, on="school_name")  
math_by_grade_df = pd.merge(math_by_grade_df, math_grade_12_grp, on="school_name")   
math_by_grade_df = math_by_grade_df.rename(columns={"math_score_x" : "11th"})
math_by_grade_df = math_by_grade_df.rename(columns={"math_score_y" : "12th"})
math_by_grade_df
#math_by_grade.groupby("grade").math_score.mean()

,school_name,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.